In [15]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/home/valentin/python_wkspce/plc_segmentation/data/dicom_info.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'image_type', 'instance_number', 'institution_name',
       'manufacturer', 'manufacturer_model_name', 'modality', 'patient_id',
       'patient_name', 'pixel_spacing', 'series_date', 'series_datetime',
       'series_instance_uid', 'series_number', 'series_time',
       'slice_thickness', 'spacing_between_slices', 'study_date',
       'study_instance_uid'],
      dtype='object')

In [9]:
df = df[df["modality"]=="PT"]
df = df.set_index("patient_id").drop("Unnamed: 0", axis=1)

In [11]:
patient_list = list(df.index)

In [12]:
patient_list.remove("PatientLC_63")  # Just one lung
patient_list.remove("PatientLC_71")  # the same as 69
patient_list.remove("PatientLC_21")  # the same as 20 

In [13]:
df = df.loc[patient_list,:]

In [14]:
df.shape

(106, 17)

In [19]:
str(df["institution_name"].values)

"['PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PTCTCHUV' 'PTCTCHUV' 'PTCTCHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'CLINIQUE DE LA SOURCE' 'CLINIQUE DE LA SOURCE' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'Clinique LA SOURCE, Lausanne'\n 'PET CT CHUV' 'Clinique LA SOURCE, Lausanne' 'PET CT CHUV' 'PET CT CHUV'\n 'CLINIQUE DE GENOLIER' 'CLINIQUE DE LA SOURCE' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV' 'PET/CT Camion' 'PET/CT Camion'\n 'Clinique LA SOURCE, Lausanne' 'PET CT CHUV'\n 'Clinique LA SOURCE, Lausanne' 'PET CT CHUV' 'PET CT CHUV' 'PET CT CHUV'\n 'PET CT CHUV' 'Clinique LA S

In [20]:
str(np.nan)

'nan'

In [28]:
df["unique_manufacturer"] = df["institution_name"].map(str) + ", " + df["manufacturer"] + ", " + df["manufacturer_model_name"] 

In [29]:
df["unique_manufacturer"]

patient_id
Case_13            PET CT CHUV, GE MEDICAL SYSTEMS, Discovery 690
Case_15            PET CT CHUV, GE MEDICAL SYSTEMS, Discovery 690
Case_7             PET CT CHUV, GE MEDICAL SYSTEMS, Discovery 690
Case_16            PET CT CHUV, GE MEDICAL SYSTEMS, Discovery 690
Case_10            PET CT CHUV, GE MEDICAL SYSTEMS, Discovery 690
                                      ...                        
PatientLC_92    CLINIQUE DE GENOLIER, Philips Medical Systems,...
PatientLC_93        nan, Philips Medical Systems, Allegro Body(C)
PatientLC_94    CLINIQUE DE GENOLIER, Philips Medical Systems,...
PatientLC_1     Clinique LA SOURCE, Lausanne, Philips Medical ...
PatientLC_2        PET CT CHUV, GE MEDICAL SYSTEMS, Discovery 690
Name: unique_manufacturer, Length: 106, dtype: object

In [30]:
df = df.sort_values(by="unique_manufacturer")

In [32]:
df.to_csv("../data/pt_manu.csv")